In [1]:
import ee
import geemap
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
import pyproj
import pandas as pd
import time
import pickle

try:
    # Initialize the library.
    ee.Initialize(project='ee-mpozdech')
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
    
# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

def image_date(image):
    time_start = ee.Date(image.get('system:time_start'))
    return time_start.format('YYYY-MM-dd').getInfo()

DATA_PATH = "../Lakes-Analysis-main/data/"

Google Earth Engine has initialized successfully!


In [2]:
coordinates = [149.35704470513303,-34.98, 149.49550371751621,-35.20614998677668]
region = ee.Geometry.Rectangle(coordinates)

#Parameters
start_date = '1984-03-16'
end_date = '2012-05-01'

pxl_fill_threshold = 40      #set % of the SAT img that must have data; 40% arbitrary set
tol = pd.Timedelta('15 day') #set how much of a difference can be between SAT and GT images

awei_threshold = 0.8         #set what constitutes a 'high-value' for the AWEI. These are usually clouds, with real water <0.6
pct_capture_threshold = 40   #set % of high-value pixel capture required (meaning % of high-value pixels also classified as water by GT) to not be droppped

#SAT data
sat_dataset = ( 
    ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
    .filterDate(start_date, end_date)
    .filterBounds(region)
    .filterMetadata('CLOUD_COVER','less_than', 50)
)

sat_dataset = sat_dataset.map(apply_scale_factors)

sat_list_num = len(sat_dataset.aggregate_array('system:index').getInfo())
print('Number of images in the SAT collection: ', sat_list_num)
sat_list = sat_dataset.toList(sat_dataset.size())

#GT data
gt_dataset = ee.ImageCollection('JRC/GSW1_4/MonthlyHistory') \
    .filterBounds(region) \
    .filterDate(start_date, end_date)

gt_list_num = len(gt_dataset.aggregate_array('system:index').getInfo())
print('Number of images in the GT collection: ', gt_list_num)
gt_list = gt_dataset.toList(gt_dataset.size())


Number of images in the SAT collection:  401
Number of images in the GT collection:  337


In [3]:
#visualizations
vis_rgb = {
    'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
    'min': 0.0,
    'max': 0.3,
}

vis_awei = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
    #'opacity' : 0.3
}

vis_gt = {
    'bands': ['water'],
    'min': 1,
    'max': 2,
    'palette': ['white', 'red'],
    'opacity' : 0.3
}

In [4]:
### GROUND TRUTH DATASET
df_gt = pd.DataFrame(columns=['date','gt_img','date_gt'])

for i in range(0,gt_list_num):
    img_id = i
    image = ee.Image(gt_list.get(img_id))
    date = image_date(image)
    
    df_gt.loc[img_id,'date'] = pd.to_datetime(date,format='%Y-%m-%d',errors='coerce',utc=True)
    df_gt.loc[img_id,'gt_img'] = image
    df_gt.loc[img_id,'date_gt'] = date # done to check if merge on date was done correctly
    
df_gt

,date,gt_img,date_gt
0,1984-04-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-04-01
1,1984-05-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-05-01
2,1984-06-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-06-01
3,1984-07-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-07-01
4,1984-08-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-08-01
...,...,...,...
332,2011-12-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-12-01
333,2012-01-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2012-01-01
334,2012-02-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2012-02-01
335,2012-03-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2012-03-01


In [5]:
### DROP GT LOW FILL

#pxl_fill_threshold = 40 #set how much of the GT img must have data; 40% arbitrary set

#df = df.drop(df[df['gt_pxl_fill'] < pxl_fill_threshold].index) # drops all rows will less than this data

In [6]:
### SAT DATASET
df = pd.DataFrame(columns=['date','sat_img','awei_img','pxl_fill','cloud_cover'])

for i in range(0,sat_list_num): #Change upper bound to be length of post-none drop df
    img_id = i
    image = ee.Image(sat_list.get(img_id)) #grabs img from dataset list
    date = image_date(image)

    df.loc[img_id,'date'] = pd.to_datetime(date,format='%Y-%m-%d',errors='coerce',utc=True)
    df.loc[img_id,'sat_img'] = image

    #compute AWEI index
    blue = image.select('SR_B1') #this doubles as the col name for water index reducer
    green = image.select('SR_B2')
    swir1 = image.select('SR_B5')
    swir2 = image.select('SR_B7')
    nir = image.select('SR_B4')
    awei_sh = blue.add(green.multiply(2.5)).subtract((nir.add(swir1)).multiply(1.5)).subtract(swir2.multiply(0.25))
    df.loc[img_id,'awei_img'] = awei_sh
    
    #compute amount of pixels which have some captured data
    band = image.select('SR_B1') #arbitrary selection of SAT band to check pixel occupancy
    none_pixels = band.mask().eq(0) #masks empty pixels (band value = 'None') with 1, otherwise 0

    #sums how many pixels in region have mask value = 1
    none_pixel_count = none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    none_count = none_pixel_count.getInfo()

    not_none_pixels = band.unmask().neq(0) #replaces all masked values (empty pixels) with 0

    #sums how many pixels in region have mask value = 1 (prior call inverted said mask)
    not_none_pixel_count = not_none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    filled_count = not_none_pixel_count.getInfo()

    df.loc[img_id,'pxl_fill'] = filled_count / (filled_count + none_count) * 100 #computes % of region which has data in it
    
    df.loc[img_id,'cloud_cover'] = image.get('CLOUD_COVER').getInfo()

#df_1 = df
df

,date,sat_img,awei_img,pxl_fill,cloud_cover
0,1987-07-16 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,39
1,1987-09-02 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,39
2,1987-09-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0
3,1987-10-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,49
4,1987-10-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,47
...,...,...,...,...,...
396,2007-01-28 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",0.042571,1
397,2007-03-17 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",0.004118,43
398,2007-04-02 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",0.052788,27
399,2008-11-14 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",13.92427,44


In [7]:
#df = df.drop(['gt_img_x','date_gt_x','gt_img_y','date_gt_y'], axis=1)
#df.reset_index(drop=True,inplace=True)
#df

In [8]:
### DROP SAT LOW FILL

df = df.drop(df[df['pxl_fill'] < pxl_fill_threshold].index) # drops all rows will less than this data
df.reset_index(drop=True,inplace=True)
#df_2 = df
df

,date,sat_img,awei_img,pxl_fill,cloud_cover
0,1987-07-16 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,39
1,1987-09-02 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,39
2,1987-09-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0
3,1987-10-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,49
4,1987-10-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,47
...,...,...,...,...,...
243,2011-03-12 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1
244,2011-03-28 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,31
245,2011-04-13 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,25
246,2011-09-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0


In [9]:
### MERGING

df['date'] = df['date'].apply(lambda x: x.to_pydatetime()) #converts timestamps to datetime
df_gt['date'] = df_gt['date'].apply(lambda x: x.to_pydatetime())

df = pd.merge_asof(left=df,right=df_gt,on='date',right_index=False,left_index=False,direction='nearest',tolerance=tol) #merge on dates
#df_3 = df
df

,date,sat_img,awei_img,pxl_fill,cloud_cover,gt_img,date_gt
0,1987-07-16 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,39,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-07-01
1,1987-09-02 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,39,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-09-01
2,1987-09-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-10-01
3,1987-10-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,49,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-10-01
4,1987-10-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,47,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-11-01
...,...,...,...,...,...,...,...
243,2011-03-12 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-03-01
244,2011-03-28 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,31,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-04-01
245,2011-04-13 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,25,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-04-01
246,2011-09-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-10-01


In [10]:
### CLEANING """accuracy"""
df_class = pd.DataFrame(columns=['tot_awei_water_pxls','tot_gt_water_pxls','awei_pxls_above_threshold','correct_pxls_above_threshold','pct_high_capture'])

for i in range(0,len(df)):
    img_index = i
    img_sat = df.loc[img_index,'sat_img'] #get Image objects from df
    img_gt = df.loc[img_index,'gt_img'].select('water')
    img_awei = df.loc[img_index,'awei_img'].select('SR_B1')
    
    awei_proj = img_sat.projection().getInfo() #gets SAT crs
    img_gt = img_gt.reproject(awei_proj['crs'],awei_proj['transform']) #uses SAT crs + transform to reproject AWEI image
    
    img_awei_water = img_awei.gt(0) #gets all values > 0 (classified as water in AWEI)

    #counts number of pixels that were marked as water in AWEI
    awei_water_count = img_awei_water.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')

    water_classified = img_gt.eq(2) #sets values = 2 (classified water in GT) to 1

    #counts number of pixels that were marked as water in GT
    gt_water_count = water_classified.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('water')
    
    df_class.loc[img_index,'tot_awei_water_pxls'] = awei_water_count.getInfo()
    df_class.loc[img_index,'tot_gt_water_pxls'] = gt_water_count.getInfo()

    awei_above_threshold = img_awei.gt(awei_threshold) #gets all pixels greater than threshold in awei img
    awei_count_above_threshold = awei_above_threshold.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')

    compare_awei_gt = water_classified.And(awei_above_threshold) #selects AWEI area where GT classified water as present
        #prior = 1 if water was correctly classified

    correctly_classified_pxls_above_threshold = compare_awei_gt.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('water')
    
    df_class.loc[img_index,'correct_pxls_above_threshold'] = correctly_classified_pxls_above_threshold.getInfo()
    df_class.loc[img_index,'awei_pxls_above_threshold'] = awei_count_above_threshold.getInfo()
    
    if awei_count_above_threshold.getInfo() != 0:
        df_class.loc[img_index,'pct_high_capture'] = correctly_classified_pxls_above_threshold.getInfo() / awei_count_above_threshold.getInfo() * 100
    else:
        df_class.loc[img_index,'pct_high_capture'] = 0

df_class

,tot_awei_water_pxls,tot_gt_water_pxls,awei_pxls_above_threshold,correct_pxls_above_threshold,pct_high_capture
0,36634.898039,28619,0,0,0
1,126603.529412,20552,47050.956863,1374,2.920238
2,28971,13325,0,0,0
3,55988.482353,13325,29075.988235,17,0.058467
4,7902.031373,23856,6472.031373,135,2.085898
...,...,...,...,...,...
243,83947,78250,0,0,0
244,80494,48984,0,0,0
245,61016.117647,48984,303.501961,35,11.532051
246,58797,13069,0,0,0


In [11]:
### MERGE df and df class on index (SAT img id)

df = pd.merge(df,df_class,left_index=True,right_index=True,how='inner')
df.reset_index(drop=True,inplace=True)
df_safe = df
df

,date,sat_img,awei_img,pxl_fill,cloud_cover,gt_img,date_gt,tot_awei_water_pxls,tot_gt_water_pxls,awei_pxls_above_threshold,correct_pxls_above_threshold,pct_high_capture
0,1987-07-16 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,39,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-07-01,36634.898039,28619,0,0,0
1,1987-09-02 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,39,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-09-01,126603.529412,20552,47050.956863,1374,2.920238
2,1987-09-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-10-01,28971,13325,0,0,0
3,1987-10-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,49,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-10-01,55988.482353,13325,29075.988235,17,0.058467
4,1987-10-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,47,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-11-01,7902.031373,23856,6472.031373,135,2.085898
...,...,...,...,...,...,...,...,...,...,...,...,...
243,2011-03-12 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-03-01,83947,78250,0,0,0
244,2011-03-28 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,31,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-04-01,80494,48984,0,0,0
245,2011-04-13 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,25,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-04-01,61016.117647,48984,303.501961,35,11.532051
246,2011-09-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-10-01,58797,13069,0,0,0


In [13]:
### ROW DROP FOR NOISY IMAGES

df = df.drop(df[(df['pct_high_capture'] < pct_capture_threshold) & (df['awei_pxls_above_threshold'] > 0)].index) # drops all rows with more than 40% of high-value pixels not captured and there were high-value pixels present
df.reset_index(drop=True,inplace=True)
df

,date,sat_img,awei_img,pxl_fill,cloud_cover,gt_img,date_gt,tot_awei_water_pxls,tot_gt_water_pxls,awei_pxls_above_threshold,correct_pxls_above_threshold,pct_high_capture
0,1987-07-16 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,39,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-07-01,36634.898039,28619,0,0,0
1,1987-09-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-10-01,28971,13325,0,0,0
2,1988-01-08 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,3,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-01-01,94,1108,0,0,0
3,1988-01-24 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-02-01,6544,2371,0,0,0
4,1988-02-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,13,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-02-01,33984,2371,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
168,2010-03-25 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,"ee.Image({\n ""functionInvocationValue"": {\n ...",2010-04-01,36245,7743,0,0,0
169,2010-12-22 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,12,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-01-01,100089.623529,62012,0,0,0
170,2011-03-12 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-03-01,83947,78250,0,0,0
171,2011-03-28 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,31,"ee.Image({\n ""functionInvocationValue"": {\n ...",2011-04-01,80494,48984,0,0,0


In [ ]:
### EXPORT

bands = ['SR_B1', 'SR_B2', 'SR_B3'] #only rgb bands for export

for i in range(0,len(df)):
    row_id = i
    sat_img = df.loc[row_id,'sat_img']
    sat_img = sat_img.select(bands) 
    awei_img = df.loc[row_id,'awei_img']
    gt_img = df.loc[row_id,'gt_img']

    capture_date = image_date(sat_img)
    awei_proj = sat_img.projection().getInfo() #gets SAT crs
    gt_repro = gt_img.reproject(awei_proj['crs'],awei_proj['transform']) #uses SAT crs + transform to reproject AWEI image

    filename_rgb = r'\Lakes-Analysis-main\data\raw\george_' + str(row_id) + '_RGB_' + capture_date + '.tif'
    filename_awei = r'\Lakes-Analysis-main\data\processed\george_' + str(row_id) + '_AWEI_' + capture_date + '.tif'
    filename_gt = r'Lakes-Analysis-main\data\ground_truth\george_' + str(row_id) + '_GT_' + capture_date + '.tif'

    geemap.ee_export_image(sat_img, filename=filename_rgb, scale=30, region=region, file_per_band=False)
    geemap.ee_export_image(awei_img, filename=filename_awei, scale=30, region=region, file_per_band=False)
    geemap.ee_export_image(gt_repro, filename=filename_gt, scale=30, region=region, file_per_band=False)

In [14]:
for i in range(1,11):
    print("The number of post-processed images with cloud cover less than or equal to",i*5,"% is",df[df['cloud_cover'] <= i*5].shape[0])

The number of post-processed images with cloud cover less than or equal to 5 % is 68
The number of post-processed images with cloud cover less than or equal to 10 % is 87
The number of post-processed images with cloud cover less than or equal to 15 % is 111
The number of post-processed images with cloud cover less than or equal to 20 % is 123
The number of post-processed images with cloud cover less than or equal to 25 % is 128
The number of post-processed images with cloud cover less than or equal to 30 % is 137
The number of post-processed images with cloud cover less than or equal to 35 % is 149
The number of post-processed images with cloud cover less than or equal to 40 % is 159
The number of post-processed images with cloud cover less than or equal to 45 % is 168
The number of post-processed images with cloud cover less than or equal to 50 % is 173


In [ ]:
### comparing images / reprojection

row_id = 17 #17 good comparison

sat_img = df.loc[row_id,'sat_img']
awei_img = df.loc[row_id,'awei_img'] 
gt_img = df.loc[row_id,'gt_img']

capture_date = image_date(sat_img)
print("captured: ",capture_date)

awei_proj = awei_img.projection().getInfo()
print("awei crs: ",awei_proj['crs'])
gt_proj = gt_img.projection().getInfo()
print("gt crs: ",gt_proj['crs'])

cloud_cover = sat_img.get('CLOUD_COVER')

print(cloud_cover.getInfo())

In [ ]:
### show AWEI map
m_awei = geemap.Map()
m_awei.set_center(149.42, -35.08,11)
m_awei.add_layer(awei_img.clip(region),vis_awei, 'awei')
m_awei

In [ ]:
### show GT map
m_gt = geemap.Map()
m_gt.set_center(149.42, -35.08,11)
m_gt.add_layer(gt_img.clip(region),vis_gt,'gt')
m_gt

In [ ]:
### show together
m_mix = geemap.Map()
m_mix.set_center(149.42, -35.08,11)
m_mix.add_layer(awei_img.clip(region),vis_awei, 'awei')
m_mix.add_layer(gt_img.clip(region),vis_gt,'gt')
m_mix

In [ ]:
### show reprojection
gt_repro = gt_img.reproject(awei_proj['crs'],awei_proj['transform'])
gt_reproj = gt_repro.projection().getInfo()
print("reprojected gt crs: ",gt_reproj['crs'])
m_mix = geemap.Map()
m_mix.set_center(149.42, -35.08,11)
m_mix.add_layer(awei_img.clip(region),vis_awei, 'awei')
m_mix.add_layer(gt_repro.clip(region),vis_gt,'gt')
m_mix